In [1]:
import pandas as pd
import numpy as np

In [2]:
import bokeh.plotting as bpl
import bokeh.models as bmo
from bokeh.palettes import d3
#bpl.output_notebook()

In [3]:
df = pd.read_csv('device_presence.log', sep=' ', names=['Day', 'Time', 'd', 'is', 'Value'])

In [4]:
df = df[['Day', 'Time', 'Value']]

In [5]:
df['Time'] = df['Time'].str[:8]

In [6]:
def nonlin(x,deriv=False):
# sigmoid function
    if(deriv==True):
        return x*(1-x)
    return 1/(1+np.exp(-x))

In [20]:
def weektime_vec(dataframe):
    in_nodes = np.zeros((len(dataframe),31))

    for i in range(len(dataframe)):
        weekday = int(dataframe.iloc[i].weekday)
        time = int(dataframe.iloc[i].time)
        # Weekday
        in_nodes[i][weekday] = 1
        # Time
        in_nodes[i][7+time] = 1
    return in_nodes

In [71]:
def train_model(df, seed=1, iterations=100):
    np.random.seed(seed)

    w0 = 2*np.random.random((31,1)) - 1
    #w1 = 2*np.random.random(())

    for i in range(iterations):
        X  = in_nodes
        #L0 = np.array(X, dtype=int) # When converting from pandas dataframe
        L0 = X
        Y  = np.array(df[['value']], dtype=int)

        L1 = nonlin(np.dot(L0, w0))

        L1_error = Y - L1

        if (i % (iterations/10) == 0):
            print('Error after iteration {i}: {error}'.format(i=i, error=np.mean(np.abs(L1_error))))

        L1_delta = L1_error * nonlin(L1, True)

        w0 += np.dot(L0.T, L1_delta)
    
    return w0

# Sample Data

In [7]:
temp = df

In [8]:
temp['time'] = temp.Time.apply(lambda x: x[:2].replace(':', ''))
temp['value'] = temp.Value.apply(lambda x: 1 if x == 'online' else 0)

temp['weekday'] = pd.to_datetime(temp['Day']).dt.dayofweek

In [73]:
in_nodes = weektime_vec(temp)

In [74]:
w0 = train_model(temp)

Error after iteration 0: 0.4874198090575956
Error after iteration 10: 0.248788756790655
Error after iteration 20: 0.24861446885529193
Error after iteration 30: 0.2473914956326398
Error after iteration 40: 0.24844287035976037
Error after iteration 50: 0.24717031602071274
Error after iteration 60: 0.24717068504537604
Error after iteration 70: 0.24717113926182405
Error after iteration 80: 0.247171712511703
Error after iteration 90: 0.24717245941597032


In [81]:
week_mat = np.zeros(7)
time_mat = np.zeros(24)

week_mat[6] = 1
time_mat[17] = 1
print('%-.15f'%nonlin(np.dot(np.hstack((week_mat, time_mat)), w0)))

1.000000000000000


In [84]:
np.savetxt('weights.csv', w0, delimiter=',', fmt='%.8f')

In [79]:
[print(i,j) for i,j in zip(w0, np.hstack((week_mat, time_mat)))]

[-297.0521543] 1.0
[-328.71112739] 0.0
[-57.02389327] 0.0
[-182.88108571] 0.0
[-23.46782598] 0.0
[117.01294384] 0.0
[177.7646838] 0.0
[-1.62962906] 0.0
[-47.83000802] 0.0
[-59.72464222] 0.0
[-46.6570164] 0.0
[-69.76081684] 0.0
[13.06904326] 0.0
[-48.11117594] 0.0
[39.90780532] 0.0
[65.73829432] 0.0
[-38.35569871] 0.0
[-100.99965486] 0.0
[-39.17021978] 0.0
[-43.75408779] 0.0
[-132.26011035] 0.0
[-137.52033879] 0.0
[-45.86413291] 0.0
[-106.20857101] 0.0
[-102.47295137] 0.0
[-110.0235141] 0.0
[47.64648445] 0.0
[119.6273682] 1.0
[110.68486027] 0.0
[101.10524003] 0.0
[40.06798562] 0.0


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Visualization

## Bokeh

In [ ]:
temp['Datetime']= temp['Day'].str.cat(temp['Time'], sep=' ')
temp['Datetime'] = pandas.to_datetime(temp['Datetime'])

In [ ]:
temp = temp[['Datetime', 'Value']]

In [ ]:
temp = temp.set_index('Datetime')

In [ ]:
temp['Value'] = temp.Value.apply(lambda x: 1 if x == 'online' else -1)

In [ ]:
temp = temp.sort_index()

In [ ]:
#temp_week = temp[:'2018-04-27']
temp_week = temp['2018-04-28':]

In [ ]:
temp_week = temp_week.resample('1H').apply(lambda x: 1 if sum(x) > 0 else -1)

In [ ]:
is_online = temp_week['Value']==1

In [ ]:
temp_week['Date'] = temp_week.index.date
temp_week['Time'] = temp_week.index.time

In [ ]:
p = bpl.figure(width=800, height=600, tools="pan,box_zoom,reset,save", y_axis_label='Day', x_axis_label='Time', y_axis_type='datetime', x_axis_type='datetime')

In [ ]:
p.circle(temp_week['Time'][is_online], temp_week['Date'][is_online], fill_color='green', size=8)
p.circle(temp_week['Time'][~is_online], temp_week['Date'][~is_online], fill_color='red', size=8)

In [ ]:
p.yaxis.formatter = bmo.formatters.DatetimeTickFormatter(days='%a')

In [ ]:
bpl.show(p)

In [ ]:
len(temp_week)

temp

In [ ]:
temp_week